In [15]:
import numpy as np
import re
import json
from tqdm import tqdm

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Concatenate

### BOW (Bag of Words) simple text encoder

In [9]:
# create a bag of words from the json
def create_bow_from_json(json_data):
    bow = []
    for _, game_data in json_data.items():
        for key in ['name', 'summary', 'genres', 'keywords']:
            if key in game_data:
                text = game_data[key]
                if isinstance(text, list):
                    text = ' '.join(text)
                # remove special characters and split into words
                words = re.findall(r'\b\w+\b', text.lower())
                for word in words:
                    if word not in bow:
                        bow.append(word)
    return bow

# create dataset for the autoencoder
def create_dataset_from_bow(dat, bow):
    X_dict = {}
    for game_id, game_data in dat.items():
        X = np.zeros(len(bow), dtype=np.float32)
        for key in ['name', 'summary', 'genres', 'keywords']:
            if key in game_data:
                text = game_data[key]
                if isinstance(text, list):
                    text = ' '.join(text)
                # remove special characters and split into words
                words = re.findall(r'\b\w+\b', text.lower())
                for word in words:
                    if word in bow:
                        X[bow.index(word)] += 1
        X_dict[game_id] = X
    return X_dict


In [10]:
# create a bag of words from the json file
with open('igdb_data_500.json', 'r') as f:
    json_data = json.load(f)
bow = create_bow_from_json(json_data)

# create the dataset
X = create_dataset_from_bow(json_data, bow)

In [16]:

# build the autoencoder model
input_dim = len(bow)  # number of unique words in the bag of words
encoding_dim = 256  # compress to 256 dims

input_bow = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_bow)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_bow, decoded)
encoder = Model(input_bow, encoded)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
X_train = np.array(list(X.values()))
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - loss: 0.6904
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.6676 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.6213
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.5457 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.4454
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.3362
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2398
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.1704
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.1230
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0930
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0782
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0678
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0650
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0614
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0603
Epoch 16/50
2/2 ━━━━━━━━━━━━━━━

In [17]:
# export the encoder model
encoder.save('game_encoder.h5')

# export the data to a JSON file
vec_json = {}
for game_id, game_data in json_data.items():
    game_bow = X[game_id] # get the bag of words vector for the game
    vec_json[game_id] = {}
    vec_json[game_id]['name'] = game_data['name']
    vec_json[game_id]['vector'] = encoder.predict(np.array([game_bow]), verbose=0)[0].tolist()
    vec_json[game_id]['img'] = game_data.get('cover_image_url', None)  # add cover image URL if available

with open('game_vecs.json', 'w') as f:
    json.dump(vec_json, f)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

### Multi-modal Encoding

In [18]:
# import image via url to process
import requests
from PIL import Image
from io import BytesIO

def url2Img(url, MAX_WIDTH=128, MAX_HEIGHT=128):
    ''' Converts an image URL to a PIL Image after resizing it to MAX_WIDTH and MAX_HEIGHT '''
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        return None
    
    img = Image.open(BytesIO(response.content))
    #resize to MAX_WIDTH x MAX_HEIGHT
    img = img.resize((MAX_WIDTH, MAX_HEIGHT), Image.Resampling.LANCZOS)
    return img

# convert the cover images to numpy arrays
def convert_images_to_arrays(dat):
    img_arrays = {}
    with tqdm(total=len(dat), desc="Processing images") as pbar:
        for game_id, game_data in dat.items():
            pbar.set_postfix({"game": game_id})
            if 'cover_image_url' in game_data and game_data['cover_image_url']:
                img = url2Img(game_data['cover_image_url'])
                if img is not None:
                    img_array = np.array(img)
                    img_arrays[game_id] = img_array
            pbar.update(1)
    return img_arrays

# create the image dataset
X_imgs = convert_images_to_arrays(json_data)
X_ratings = {game_id: game_data.get('rating', 0) for game_id, game_data in json_data.items()}

Processing images: 100%|██████████| 498/498 [01:16<00:00,  6.52it/s, game=11137] 


In [22]:
# --- Parameters ---
bow_dim = len(bow)        # input size of BoW
image_shape = (128, 128, 3)
encoding_dim = 256     # final embedding size

# --- Inputs ---
text_input = Input(shape=(bow_dim,), name='text_input')
image_input = Input(shape=image_shape, name='image_input')
float_input = Input(shape=(1,), name='float_input')

# --- Text Encoder (BoW -> Dense) ---
text_encoded = Dense(512, activation='relu')(text_input)

# --- Image Encoder (Simple CNN) ---
x = Conv2D(32, (3,3), activation='relu')(image_input)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
image_encoded = Dense(256, activation='relu')(x)

# --- Float Encoder (Dense) ---
float_encoded = Dense(32, activation='relu')(float_input)

# --- Merge and Final Encoding ---
merged = Concatenate()([text_encoded, image_encoded, float_encoded])
final_encoding = Dense(encoding_dim, activation='relu', name='embedding')(merged)

# --- Model ---
multi_input_encoder = Model(inputs=[text_input, image_input, float_input],
                            outputs=final_encoding)
multi_input_encoder.compile(optimizer='adam', loss='mse')

In [ ]:
# train the model
multi_input_encoder.compile(optimizer='adam', loss='mse')
X_train_image = np.array(X_imgs.values())
X_train_float = np.array([X_ratings[game_id] for game_id in json_data.keys() if game_id in X_ratings])
X_train_text = np.array([X[game_id] for game_id in json_data.keys() if game_id in X])

print(f"Training with {len(X_train_text)} text samples, {len(X_train_image)} image samples, and {len(X_train_float)} float samples.")
print(f"Text shape: {X_train_text.shape}, Image shape: {X_train_image.shape}, Float shape: {X_train_float.shape}")
multi_input_encoder.fit([X_train_text, X_train_image, X_train_float],
                        X_train_text,  # using text as target for simplicity
                        epochs=50, batch_size=256, shuffle=True)


495


TypeError: len() of unsized object

In [ ]:
# export the multi-input encoder model
multi_input_encoder.save('multi_input_encoder.h5')

# export the data to a JSON file
multi_vec_json = {}
for game_id, game_data in json_data.items():
    multi_vec_json[game_id] = {
        "name": game_data["name"],
        "vector": multi_input_encoder.predict(game_data["input"]).tolist(),
        "img": game_data.get('cover_image_url', None)  # add cover image URL if available
    }

with open('game_multivecs.json', 'w') as f:
    json.dump(multi_vec_json, f)